In [2]:
import laspy
import open3d as o3d
import numpy as np
from pyntcloud import PyntCloud
import pylas
import pptk
from sklearn.decomposition import PCA
import os
import matplotlib as plt



ImportError: dlopen(/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/open3d/cpu/pybind.cpython-38-darwin.so, 0x0002): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: <3DD86FDF-C162-3CA9-BE4C-3A6B0DCEBF39> /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/open3d/cpu/pybind.cpython-38-darwin.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file, not in dyld cache)

# Preprossessing

In [1]:
#reading the las file.
path = "data.las"
las = laspy.read(path)

# #Coordinates of points
coords = np.vstack((las.x, las.y, las.z)).transpose()
colors = np.vstack((las.red, las.green, las.blue)).transpose()
print(coords.shape)



NameError: name 'laspy' is not defined

## Subset was selected from the PPTK - Viewer

<b> To select subset, run step 1. </b> <br> - Hold [CTRL] and select area with [LMB]. <br> - To clear all selected press [RMB], or Use [shift] + [CTRL] and mark to remove selected points.
<br>
<br>
<b> Dont close the Viewer and run step 2. </b>
<br> After step.2 is done, the Viewer is safe to exit and the points are stored in the variable.

### Step 1 - Visualize and select

In [277]:
v = pptk.viewer(coords)
v.set(point_size=0.005)

### Step 2 - get selected points id and find the points

In [15]:
points_id = v.get('selected')
points = coords[points_id]

## Using open3D tool for noise filtering

In [17]:
pcd_sub = o3d.geometry.PointCloud()
pcd_sub.points = o3d.utility.Vector3dVector(points)
cl, ind = pcd_sub.remove_radius_outlier(nb_points=10, radius=5)

def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

display_inlier_outlier(pcd_sub, ind)

Showing outliers (red) and inliers (gray): 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


## Saving subset without noise as .ply file

In [18]:
inlier_cloud = pcd_sub.select_by_index(ind)
o3d.io.write_point_cloud("data.ply", inlier_cloud)

True

## Loading the subset

In [19]:
cloud = PyntCloud.from_file("data.ply")
pcd_sub = o3d.io.read_point_cloud("data.ply")


## Voxelization

In [930]:
voxel_size = 5  #x*y*z box size
voxelgrid_id = cloud.add_structure("voxelgrid", size_x=voxel_size, size_y=voxel_size, size_z=voxel_size)
voxelgrid = cloud.structures[voxelgrid_id]
new_cloud = cloud.get_sample("voxelgrid_nearest", voxelgrid_id=voxelgrid_id, as_PyntCloud=True)

print(f"Number of points {len(cloud.points)}, number of voxels {voxelgrid.n_voxels}")
occupied_voxels = np.unique(voxelgrid.voxel_n)
print("Occupied voxels:", len(occupied_voxels))

Number of points 3419462, number of voxels 2016252
Occupied voxels: 18819


## Visualize voxelgrid
The density mode will return a 3D array with a value normalized between 0 and 1 representing the percentage of points that each voxel contains.

In [931]:
#open3d
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd_sub, voxel_size=voxel_size)
o3d.visualization.draw_geometries([voxel_grid])

voxelgrid.plot(d=3, mode="density", cmap="OrRd")

ImportError: matplotlib is required for non-binary plotting

## PCA

In [932]:
pca = PCA(n_components=3) #tre dimensjoner
pcd_array = np.asarray(cloud.points)
print(pcd_array[0])

[5.76801470e+05 7.03377685e+06 4.49300000e+01 5.99532000e+05
 3.04353000e+05 1.42541100e+06]


## Feature selection

In [933]:
features = np.zeros((occupied_voxels.shape[0], 11))

In [934]:
def standard_deviation_from_plane(points):
        # Plane calculation mostly taken from: 
        # https://gist.github.com/amroamroamro/1db8d69b4b65e8bc66a6

        # The centroid
        centroid = np.sum(points, axis=0)/points.shape[0]

        # Centered data
        c_data = points - centroid
        A = np.c_[c_data[:,0], c_data[:,1], np.ones(c_data.shape[0])]

        # Plane coefficients
        C,_,_,_ = np.linalg.lstsq(A, c_data[:,2], rcond=None)
        #       Z = C[0]*X + C[1]*Y + C[2]
        # =>    C[0]*X + C[1]*Y -1*Z + C[2] = 0

        # Distances from the plane
        distances = np.abs(C[0]*c_data[:,0] + C[1]*c_data[:,1] -1*c_data[:,2] + C[2]) / np.sqrt(C[0]**2 + C[1]**2 + 1)

        # Normalize distances
        distances /= voxel_size

        # Return the standard deviation
        return np.std(distances)


# Finding features for each voxel

In [935]:
for i, voxel in enumerate(occupied_voxels):

    volex_i = np.where(voxelgrid.voxel_n == voxel)
    voxel_points = pcd_array[volex_i]
    # print(f"voxel points {voxel_points}")
    
    #voxel have to contain enough points
    if len(volex_i[0]) > 5*voxel_size: 
        #Finding eigenvalue and vector
        pca.fit(voxel_points)
        eigenval = pca.explained_variance_
        eigenval_norm = pca.explained_variance_ratio_
        eigenvectors = pca.components_

        #Finding normals angle for slope-classificaiton
        normal_vec = eigenvectors[2]
        normal_vec = normal_vec[:3]
        z_normal = np.array([0,0,1])
        angle = np.degrees(np.arccos(np.dot(normal_vec, z_normal ) / (np.linalg.norm(normal_vec) * np.linalg.norm(z_normal))))
        if angle > 90:
            angle = 180 - angle 

        #Get lambda1, lambda2, lambda3 from eigenvalue
        l1, l2, l3 = eigenval_norm

        L = (l1 - l2) / l1                                                      # Linearity
        P = (l2 - l3) / l1                                                      # Planarity
        S = l3 / l1                                                             # Scattering, sphericity
        O = (l1 * l2 * l3) ** (1 / 3)                                           # Omnivariance
        sum_ev = eigenval[0] + eigenval[1] + eigenval[2]                        # Sum of eigenvalues
        sum_ev_norm = l1 + l2 + l3                                              # Sum of un-normalized eigenvalues
        A = (l1 - l3) / l1                                                      # Anisotropy
        E = -1 * ((l1 * np.log(l1)) + (l2 * np.log(l2)) + (l3 * np.log(l3)))    # Eigenentropy
        change_curvature = l3 / sum_ev_norm                                     # Change of curvature

        
        #point spread in z axis
        z_range = (np.max(voxel_points[:,2]) - np.min(voxel_points[:,2])) / voxel_size

        #the standard deviation from plane
        plane_std = standard_deviation_from_plane(voxel_points)

        feature = [L, P, S, O, sum_ev_norm, A, E, change_curvature, z_range, plane_std, angle]
        features[i] = feature

# Selecting points for known classes to find ground truth of class features

# Building selection

In [890]:
v = pptk.viewer(pcd_sub.points)
v.set(point_size=0.005)

In [891]:
building = v.get('selected')
builing_points = pcd_array[building]
building_voxel_id = voxelgrid.query(builing_points)
building_voxel_id = np.unique(building_voxel_id)
building_voxel_id

array([1356704, 1356829, 1356830, 1372579, 1372580, 1372705, 1372706,
       1372831, 1372832, 1372833, 1372958, 1372959, 1388581, 1388582,
       1388707, 1388708, 1388709, 1388834, 1388835, 1404583, 1404584,
       1404710, 1404711, 1404836, 1404837, 1420585, 1420586, 1420712,
       1420713, 1420838, 1436587, 1436588, 1436713, 1436714, 1436715,
       1436839, 1436840], dtype=int64)

## Finding feature indicies by finding voxel in occupied voxels

In [907]:
building_indecies = []
for i, voxel in enumerate(occupied_voxels):
    for build_voxel in building_voxel_id:
        if voxel == build_voxel:
            building_indecies.append(i)

print(building_indecies)

building_indecies = [
 15140, 15141, 15143, 15144,
15337, 15338, 15340, 15341,
15543, 15544, 15546,
]



[14717, 14718, 14719, 14921, 14922, 14924, 14925, 14926, 14927, 14928, 14930, 14931, 15137, 15138, 15139, 15140, 15141, 15143, 15144, 15335, 15336, 15337, 15338, 15340, 15341, 15541, 15542, 15543, 15544, 15546, 15761, 15762, 15763, 15764, 15765, 15766, 15767]


## Calulating mean features for buildings. <br>

In [908]:
# Printing feature for each voxel
for v_i in building_indecies:
    feature = features[v_i]
    L = feature[0]
    P = feature[1] 
    S = feature[2] 
    O = feature[3]
    sumEvN = feature[4]
    A = feature[5] 
    Eigentrhopy = feature[6] 
    change_curvature =feature[7]
    z_range = feature[8]
    st_plane = feature[9]
    angle = feature[10]
    
    # print(f" Linear {L}, Planarity {P}, Scattering {S}, Omnivariance {O}, Anisotropy {A}, Eigenthropy {Eigentrhopy}, change_curvature {change_curvature}, z_range {z_range}, st_plane {st_plane} ")
    # print(f"angle: {angle}")


#VISUALIZE SELECTEDBUIILDING VOXEL POINTS
# # Initialize the point arrays
selected_building_points = np.zeros((1,3))
for i in building_indecies:
    voxel = occupied_voxels[i]
    tmp = np.where(voxelgrid.voxel_n == voxel)
    voxel_points = pcd_array[tmp]
    selected_building_points = np.append(selected_building_points, voxel_points[:,:3], axis=0)

# # Delete the initial zeros
selected_building_points = np.delete(selected_building_points, 0, axis=0)

building = o3d.geometry.PointCloud()
building.points = o3d.utility.Vector3dVector(selected_building_points)
building.paint_uniform_color([0, 1, 0])
o3d.visualization.draw_geometries([building])

## Ground truth

In [909]:
## Metode
building_mean_feature = np.mean([features[v_i] for v_i in building_indecies], axis=0)

# mine
# building_mean_feature = np.array([9.19687852e-01, 1.92624628e-02, 2.22615534e-03, 6.33922124e-03, 9.41134298e-01, 9.38950315e-01, 2.45037245e-02, 1.23423423e-03, 3.85882353e-01, 3.40600664e-02, 6.75481204e+01])

# Data Sarun
# building_mean_feature = np.array([3.16798368e-01, 5.91370207e-01, 3.18314249e-02, 1.06457223e-01, 9.40000000e-01, 9.08168575e-01, 6.65824570e-01, 1.77401593e-02, 4.70950000e-01, 2.53849077e-02, 2.21350506e+01])

# Data Alfred
# building_mean_feature = np.array([0.33163354789966903, 0.5804872035663875, 0.08787924853394344, 0.1970603392467401, 11.058980588099054, 0.9121207514660565, 0.7935565870483143, 0.046663981433086914, 0.6151249885559082, 0.060233117960162216, 0])

# fra excel
# building_mean_feature = np.array([])


print(building_mean_feature)

[5.45454532e-01 1.04073900e-08 3.11057314e-09 2.81015703e-06
 5.45454545e-01 5.45454542e-01 3.12330678e-07 3.11057304e-09
 4.80363636e-01 1.56070880e-02 4.74231349e+01]


## Classifying builing voxels based on ground truth

In [971]:
building_voxels = []

for i, feature in enumerate(features):
    L = np.abs(feature[0] - building_mean_feature[0])
    P = np.abs(feature[1] - building_mean_feature[1])
    S = np.abs(feature[2] - building_mean_feature[2])
    O = np.abs(feature[3] - building_mean_feature[3])
    sumEvN = np.abs(feature[4] - building_mean_feature[4])
    A = np.abs(feature[5] - building_mean_feature[5])
    Eigentrhopy = np.abs(feature[6] - building_mean_feature[6])
    change_curvature = np.abs(feature[7] - building_mean_feature[7])

    Lin = feature[0]
    plan = feature[1]
    scat = feature[2]
    sumE = feature[4]
    an = feature[5]
    cc = feature[7]
    h = feature[8]
    stp = feature[9]

    z_range = np.abs(feature[8] - building_mean_feature[8])
    st_plane =np.abs(feature[9] - building_mean_feature[9])
    angle = feature[10]
    #st 0,05 o < 0.1
    #Sarun
    #if L < 0.7 and P < 0.3 and S < 0.06 and A < 0.12 and change_curvature > 0.005 and z_range < 0.3 and st_plane < 0.035 and angle > 18 and angle < 40:
    #  building_voxels.append(i) 
    #if L < 0.7 and P < 0.3 and S < 0.08 and A < 0.12 and change_curvature > 0.001 and z_range < 0.33 and st_plane < 0.04 and angle > 15 and angle < 60: 
    #  building_voxels.append(i)
    
    #bpa excel
    cc = feature[7]
    h = feature[8]
    angle = feature[10]

    if 0.1<h<0.7 and angle<70 and cc<0.00001:
      building_voxels.append(i)
    # cc<0.00001 kanskje bra for å skille tree og hus
    # h<0.45 and h>0.2
    # stp>0.01 fjerner mye bakke, ligger i kryssningspunkt
    # 0.01<stp<0.04 ganske ok
    # an < 0.9 tar ikke hele taket
      

print(len(building_voxels))

1129


## Visualizing classified buildings

In [972]:
# Initialize the point arrays
building_points = np.zeros((1,3))

for i in building_voxels:
    voxel = occupied_voxels[i]
    tmp = np.where(voxelgrid.voxel_n == voxel)
    voxel_points = pcd_array[tmp]
    building_points = np.append(building_points, voxel_points[:,:3], axis=0)

# # Delete the initial zeros
building_points = np.delete(building_points, 0, axis=0)

In [973]:
building = o3d.geometry.PointCloud()
building.points = o3d.utility.Vector3dVector(building_points)
building.paint_uniform_color([0, 1, 0])
o3d.visualization.draw_geometries([building])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [918]:
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd_sub, voxel_size=voxel_size)

voxel_grid3 = o3d.geometry.VoxelGrid.create_from_point_cloud(building, voxel_size=voxel_size)
o3d.visualization.draw_geometries([voxel_grid, voxel_grid3])

# Tree selection

## Selecting trees

In [626]:
v = pptk.viewer(pcd_sub.points)
v.set(point_size=0.005)

In [627]:
tree = v.get('selected')
tree_points = pcd_array[tree]
tree_voxel_id = voxelgrid.query(tree_points)
len(tree_voxel_id)
tree_voxel_id = np.unique(tree_voxel_id)
tree_voxel_id

array([1256158, 1256159, 1256285, 1272034, 1272160, 1272161, 1272162,
       1272163, 1272286, 1272287, 1272288, 1272289, 1272412, 1272413,
       1287910, 1287911, 1288036, 1288037, 1288038, 1288162, 1288163,
       1288164, 1288165, 1288288, 1288289, 1288290, 1288291, 1288414,
       1288415, 1288416, 1288417, 1288541, 1288542, 1288543, 1303786,
       1303912, 1303913, 1303914, 1304038, 1304039, 1304040, 1304041,
       1304164, 1304165, 1304166, 1304291, 1304292, 1304293, 1304417,
       1304418, 1304419, 1304543, 1304544, 1304545, 1304670, 1304671,
       1319662, 1319788, 1319789, 1319790, 1319914, 1319915, 1319916,
       1319917, 1320041, 1320042, 1320043, 1320167, 1320168, 1320169,
       1320293, 1320294, 1320295, 1320419, 1320420, 1320421, 1320422,
       1320545, 1320546, 1320547, 1320548, 1335790, 1335791, 1335792,
       1335917, 1335918, 1335919, 1336043, 1336044, 1336045, 1336169,
       1336170, 1336295, 1336296, 1336421, 1336422, 1336423, 1336549,
       1351919, 1351

## Finding the Voxel indecies

In [628]:
# #finding voxel indices in occupied_voxel
tree1_indecies = []
for i, voxel in enumerate(occupied_voxels):

    for t1 in tree_voxel_id:
        if voxel == t1:
            tree1_indecies.append(i)
#tree1_indecies = [4, 5, 6, 7, 8, 9, 10, 11, 42, 43, 44, 45, 46, 47, 48, 49, 50, 174, 175, 176]
#tree1_indecies = [6, 44, 45]




## Features stats and visualization of selected voxels

In [629]:
selected_tree_points = np.zeros((1,3))
for i in tree1_indecies:
    voxel = occupied_voxels[i]
    tmp = np.where(voxelgrid.voxel_n == voxel)
    voxel_points = pcd_array[tmp]
    selected_tree_points = np.append(selected_tree_points, voxel_points[:,:3], axis=0)

# # Delete the initial zeros
selected_building_points = np.delete(selected_tree_points, 0, axis=0)

building = o3d.geometry.PointCloud()
building.points = o3d.utility.Vector3dVector(selected_building_points)
building.paint_uniform_color([0, 1, 0])
o3d.visualization.draw_geometries([building])

## Ground truth

In [630]:
tree1_mean_feature = np.mean([features[v_i] for v_i in tree1_indecies], axis=0)


#Mine
# tree1_mean_feature = np.array([7.99025145e-01, 1.59478541e-02, 3.20881882e-03, 7.33456574e-03, 8.17651873e-01, 8.14972999e-01, 2.64944870e-02, 1.85358220e-03, 4.56363636e-01, 4.70674218e-02, 20 ])
# angle: 4.99243650e+01

#Sarun
# tree1_mean_feature = np.array([0.23167487, 0.45972011, 0.11485502, 0.15384777, 0.80625, 0.69139498, 0.66793328, 0.05757029, 0.45966295, 0.05032597, 20.97007081])

#Alfred
# tree1_mean_feature = np.array([0.3241482310307115, 0.33303603180630464, 0.3428157371629839, 0.28171727165415233, 12.385173027039873, 0.6571842628370163, 0.9736421385613483, 0.16015040965855912, 0.8417498588562011, 0.10096039720609315, 0])

# fra excel

# tree1_mean_feature = np.array([0.8857016063, 0.0007450292, 0.0017620378, 0.0021789505, 0.8878655635, 0.8864466358, 0.0067251914, 0.0010698856, 0.6482156075, 0.0741382717, 59.9002451250])


print(tree1_mean_feature)

[8.89830001e-01 2.74336982e-03 2.16347135e-03 2.60931148e-03
 8.94734396e-01 8.92573371e-01 8.56252093e-03 1.20086706e-03
 6.86070175e-01 8.71463717e-02 4.92788267e+01]


## Classification of trees

In [979]:
tree_voxels = []
for i, feature in enumerate(features):
    L = np.abs(feature[0] - tree1_mean_feature[0])
    P = np.abs(feature[1] - tree1_mean_feature[1])
    S = feature[2]
    O = feature[3]
    sumEvN = np.abs(feature[4] - tree1_mean_feature[4])
    A = np.abs(feature[5] - tree1_mean_feature[5])
    Eigentrophy = np.abs(feature[6] - tree1_mean_feature[6])

    E = feature[6]

    change_curvature = np.abs(feature[7] - tree1_mean_feature[7])
    z_range = feature[8]
    st_plane = feature[9]
    angle = feature[10]
    #sarun
    #if L < 0.2 and P < 0.2 and angle > 10 and z_range > 0.2 and S > 0.07 and O > 0.2:
    #    tree_voxels.append(i)
    #min
    #if L < 0.7 and P < 0.4 and A < 0.4 and Eigentrophy < 0.8 and z_range > 0.2 and angle > 10:
    #    tree_voxels.append(i)

    #pba excel, ganske fornøyd
    z_range = feature[8]
    S = feature[2]
    E = feature[6]
    if z_range > 0.5 and S>0.00000001 and E > 0.0000005:
        tree_voxels.append(i)
    #L<0.110169985
    # S>0.00000001 bra
    # E > 0.0000005 bra
    # angle>65 tar for sæ mye
    # z_range > 0.5 bra
print(len(tree_voxels))

214


## Finding point cloud of classified voxels

In [728]:
# # Initialize the point arrays
tree_points = np.zeros((1,3))

for i in tree_voxels:
    voxel = occupied_voxels[i]
    tmp = np.where(voxelgrid.voxel_n == voxel)
    voxel_points = pcd_array[tmp]
    tree_points = np.append(tree_points, voxel_points[:,:3], axis=0)

# #Delete first point
tree_points = np.delete(tree_points, 0, axis=0)

## Visualizing trees

In [729]:
tree_cloud = o3d.geometry.PointCloud()
tree_cloud.points = o3d.utility.Vector3dVector(tree_points)
tree_cloud.paint_uniform_color([0, 1, 0])
o3d.visualization.draw_geometries([tree_cloud])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


# Terrain selection

## Selecting terrain

In [987]:
v = pptk.viewer(pcd_sub.points)
v.set(point_size=0.005)

In [603]:
terrain_selected = v.get('selected')
terrain_points = pcd_array[terrain_selected]
terrain_voxel_id = voxelgrid.query(terrain_points)
terrain_voxel_id = np.unique(terrain_voxel_id)
print(terrain_voxel_id)

[466893 467019 467145 467271 467397 467523 467524 467649 467775 482770
 482895 482896 483021 483022 483147 483148 483273 483274 483399 483400
 483525 483526 483651 483652 483777 483778 483903 483904 498646 498772
 498898 499024 499150 499276 499402 499528 499654 499780 499906 514397
 514522 514523 514648 514649 514774 514775 514900 514901 515026 515027
 515152 515153 515278 515279 515404 515530 515656 530399 530525 530651
 530777 530903 531028 531029 531154 531155 531280 531281 531406 531532
 546275 546401 546527 546653 546779 546904 546905 547030 547031 547156
 547157 547282 547283 547408 562403 562529 562655 562781 562907 563032
 563033 563158 563159 563284 563285 563410 578909 579035 579161 579287]


## Finding terrain voxel and calulating mean features for ground truth

In [604]:
# #finding voxel indices in occupied_voxel
terrain_indecies = []
for i, voxel in enumerate(occupied_voxels):

    for t1 in terrain_voxel_id:
        if voxel == t1:
            terrain_indecies.append(i)


In [607]:
# [Linearity, Planarity, Scatteing, Omnivariance, Sum Eigenvalues, Anistrophy, eignetrophy, Change of curvature, z-range, StD from plane, angle]

terrain_mean_feature = np.mean([features[v_i] for v_i in terrain_indecies], axis=0)
print(terrain_mean_feature)
#Sarun
#terrain_mean_feature = np.array([1.39090066e-01, 7.76421146e-01, 2.34028906e-03, 4.50057163e-02, 9.17851501e-01, 9.15511212e-01, 6.27657881e-01, 1.29142198e-03, 1.59202212e-01, 5.77090346e-03, 8.09390507e+00])

# ##GROUND TRUTH
#min
#terrain_mean_feature = np.array([9.99999984e-01, 9.56474653e-09, 6.62514249e-09, 4.67089482e-06, 1.00000000e+00, 9.99999993e-01, 4.37086500e-07, 6.62514232e-09, 1.72842105e-01, 2.58821045e-03, 8.63182586e+01])

#Calculated in excel
#terrain_mean_feature = np.array([0.8841605248, 0.0039894370, 0.0008040950, 0.0018882005, 0.8888215600, 0.8881499615, 0.0066831882, 0.0004652851, 0.2588250910, 0.0217988055, 78.6775515250])




[9.39983927e-01 9.40591051e-06 6.66713625e-06 1.96987694e-04
 9.39999969e-01 9.39993333e-01 2.19764936e-04 6.66518356e-06
 1.85800000e-01 8.73667397e-03 7.47055445e+01]


## Classifying terrain

In [982]:
terrain_voxels = []
for i, feature in enumerate(features):
    L = np.abs(feature[0] - terrain_mean_feature[0])
    P = np.abs(feature[1] - terrain_mean_feature[1])
    S = np.abs(feature[2] - terrain_mean_feature[2])
    O = np.abs(feature[3] - terrain_mean_feature[3])
    sumEvN = np.abs(feature[4] - terrain_mean_feature[4])
    A = np.abs(feature[5] - terrain_mean_feature[5])
    Eigentrhopy = np.abs(feature[6] - terrain_mean_feature[6])
    change_curvature = np.abs(feature[7] - terrain_mean_feature[7])
    z_range = np.abs(feature[8] - terrain_mean_feature[8])
    st_plane = np.abs(feature[9] - terrain_mean_feature[9])
    angle = feature[10]
    # Sarun
    #score = np.sum(L+P+S+O+change_curvature+z_range)
    #if score < 0.5: 
    #    terrain_voxels.append(i)

    #min
    # if z_range < 0.3 and st_plane <0.03 and P < 0.000001 and S < 0.000001 and O < 0.00001 : 
    #    terrain_voxels.append(i)


    #from excel
    scat = feature[2]
    h = feature[8]

    if scat<0.00001 and h<0.22: 
        terrain_voxels.append(i)
    #L > 0.08696396
    
print(len(terrain_voxels))


9216


## Finding cloud point and visualizing cloud points

In [981]:
# Time: 14 000p => 4min
# # # Initialize the point arrays
terrain_points = np.zeros((1,3))

for i in terrain_voxels:
    voxel = occupied_voxels[i]
    tmp = np.where(voxelgrid.voxel_n == voxel)
    voxel_points = pcd_array[tmp]
    terrain_points = np.append(terrain_points, voxel_points[:,:3], axis=0)

# #VISUALIZE
terrain_points = np.delete(terrain_points, 0, axis=0)
terrain_cloud = o3d.geometry.PointCloud()
terrain_cloud.points = o3d.utility.Vector3dVector(terrain_points)
terrain_cloud.paint_uniform_color([0, 1, 0])
o3d.visualization.draw_geometries([terrain_cloud])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


# Classifying all the Voxels
### with the ground truth and parameters found above.

In [101]:
building_mean_feature = np.array([3.16798368e-01, 5.91370207e-01, 3.18314249e-02, 1.06457223e-01, 9.40000000e-01, 9.08168575e-01, 6.65824570e-01, 1.77401593e-02, 4.70950000e-01, 2.53849077e-02, 2.21350506e+01])
tree1_mean_feature = np.array([0.23167487, 0.45972011, 0.11485502, 0.15384777, 0.80625, 0.69139498, 0.66793328, 0.05757029, 0.45966295, 0.05032597, 20.97007081])
terrain_mean_feature = np.array([1.39090066e-01, 7.76421146e-01, 2.34028906e-03, 4.50057163e-02, 9.17851501e-01, 9.15511212e-01, 6.27657881e-01, 1.29142198e-03, 1.59202212e-01, 5.77090346e-03, 8.09390507e+00]) 

## Classifying

In [983]:
terrain_voxels = []
tree_voxels = []
building_voxels = []
unclassified_voxels = []

for i, feature in enumerate(features):
    cc = feature[7]
    h = feature[8]
    angle = feature[10]

    if 0.1<h<0.7 and angle<70 and cc<0.00001:
      building_voxels.append(i)
      continue
##  TODO: make function
    #BUILDING CLASSIFICATION
    #L3 = np.abs(feature[0] - building_mean_feature[0])
    #P3 = np.abs(feature[1] - building_mean_feature[1])
    #S3 = np.abs(feature[2] - building_mean_feature[2])
    #O3 = np.abs(feature[3] - building_mean_feature[3])
    #sumEvN3 = np.abs(feature[4] - building_mean_feature[4])
    #A3 = np.abs(feature[5] - building_mean_feature[5])
    #Eigentrhopy3 = np.abs(feature[6] - building_mean_feature[6])
    #change_curvature3 = np.abs(feature[7] - building_mean_feature[7])
    #z_range3 = np.abs(feature[8] - building_mean_feature[8])
    #st_plane3 =np.abs(feature[9] - building_mean_feature[9])
    #angle3 = feature[10]

    #if L3 < 0.8 and P3 < 0.5 and S3 < 0.08 and A3 < 0.12 and change_curvature3 > 0.01 and z_range3 < 0.3 and st_plane3 < 0.035 and angle3 > 10 and angle3 < 40: 
    #    building_voxels.append(i)
    #    continue

    

##  TODO: make function
    z_range = feature[8]
    S = feature[2]
    E = feature[6]
    if z_range > 0.5 and S>0.00000001 and E > 0.0000005:
        tree_voxels.append(i)
        continue
    #TREE CLASSIFICATION
    #L2 = np.abs(feature[0] - tree1_mean_feature[0])
    #P2 = np.abs(feature[1] - tree1_mean_feature[1])
    #S2 = feature[2]
    #O2 = feature[3]
    #sumEvN2 = np.abs(feature[4] - tree1_mean_feature[4])
    #A2 = np.abs(feature[5] - tree1_mean_feature[5])
    #Eigentrhopy2 = np.abs(feature[6] - tree1_mean_feature[6])
    #change_curvature2 = np.abs(feature[7] - tree1_mean_feature[7])
    #z_range2 = feature[8]
    #st_plane2 = feature[9]
    #angle2 = feature[10]
    
    #if L2 < 0.4 and P2 < 0.4 and angle2 > 10 and z_range2 > 0.1 and st_plane2 > 0.04 and S2 > 0.03 and O2 > 0.2:
    #    tree_voxels.append(i)
    #    continue

#  TODO: make function
    #TERRAIN CLASSIFICATION
    scat = feature[2]
    h = feature[8]

    if scat<0.00001 and h<0.22: 
        terrain_voxels.append(i)
        continue

    #L1 = np.abs(feature[0] - terrain_mean_feature[0])
    #P1 = np.abs(feature[1] - terrain_mean_feature[1])
    #S1 = np.abs(feature[2] - terrain_mean_feature[2])
    #O1 = np.abs(feature[3] - terrain_mean_feature[3])
    #sumEvN1 = np.abs(feature[4] - terrain_mean_feature[4])
    #A1 = np.abs(feature[5] - terrain_mean_feature[5])
    #Eigentrhopy1 = np.abs(feature[6] - terrain_mean_feature[6])
    #change_curvature1 = np.abs(feature[7] - terrain_mean_feature[7])
    #z_range1 = np.abs(feature[8] - terrain_mean_feature[8])
    #st_plane1 = np.abs(feature[9] - terrain_mean_feature[9])
    #angle1 = feature[10]
    #
    #score = np.sum(L1+P1+S1+O1+change_curvature1+z_range1)
    #if score < 1: 
    #    terrain_voxels.append(i)
    #    continue   

    #Adding indecies to unclassified list
    unclassified_voxels.append(i)
    

## Visualizing results

In [984]:
print(f"Terrain voxel classified {len(terrain_voxels)}, Tree voxel classified {len(tree_voxels)}, Building voxel classified {len(building_voxels)}, voxel unclassified {len(unclassified_voxels)}")

Terrain voxel classified 9178, Tree voxel classified 190, Building voxel classified 1129, voxel unclassified 8322


In [985]:
def voxel_to_cloudpoints(x_voxels, occupied_voxels, pcd_array, voxelgrid):
    x_points = np.zeros((1,3))

    for i in x_voxels:
        voxel = occupied_voxels[i]
        tmp = np.where(voxelgrid.voxel_n == voxel)
        voxel_points = pcd_array[tmp]
        x_points = np.append(x_points, voxel_points[:,:3], axis=0)

    x_points = np.delete(x_points, 0, axis=0)
    return x_points

### NOTE: this cell takes around 5 min to run

In [986]:
# Initialize the point arrays
terrain_points = voxel_to_cloudpoints(terrain_voxels, occupied_voxels, pcd_array, voxelgrid)
tree_points = voxel_to_cloudpoints(tree_voxels, occupied_voxels, pcd_array, voxelgrid)
building_points = voxel_to_cloudpoints(building_voxels, occupied_voxels, pcd_array, voxelgrid)

#VISUALIZE
building = o3d.geometry.PointCloud()
building.points = o3d.utility.Vector3dVector(building_points)
building.paint_uniform_color([1, 0, 0])

terrain = o3d.geometry.PointCloud()
terrain.points = o3d.utility.Vector3dVector(terrain_points)
terrain.paint_uniform_color([0, 1, 0])

tree = o3d.geometry.PointCloud()
tree.points = o3d.utility.Vector3dVector(tree_points)
tree.paint_uniform_color([0, 0, 1])

# o3d.visualization.draw_geometries([building, terrain, tree])

# Voxelgrid
voxel_grid1 = o3d.geometry.VoxelGrid.create_from_point_cloud(tree, voxel_size=voxel_size)
voxel_grid2 = o3d.geometry.VoxelGrid.create_from_point_cloud(terrain, voxel_size=voxel_size)
voxel_grid3 = o3d.geometry.VoxelGrid.create_from_point_cloud(building, voxel_size=voxel_size)
o3d.visualization.draw_geometries([voxel_grid1, voxel_grid2, voxel_grid3])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


# Evaluations

In [ ]:
# v = pptk.viewer(pcd_sub.points)
# v.set(point_size=0.005)

In [ ]:
# building = v.get('selected')
# builing_points = pcd_array[building]
# building_voxel_id = voxelgrid.query(builing_points)
# building_voxel_id = np.unique(building_voxel_id)
# building_voxel_id


In [ ]:
terreng = np.array([ 440911,  441074,  441075,  441238,  467970,  468134,  468135, 468298,  543415,  543416,  543580,  570476, 1394403, 1394567, 1409502, 1421463, 1421627, 1436398, 1436562, 1436726, 1463458,
       1463622, 1463623, 1463786, 2131254, 2131418, 2131582, 2719204, 2719368, 3055881, 3055882, 3056045, 3056209, 3082941, 3082942, 3083105, 3083269, 3110165, 3110329, 3155275])

tree = np.array([214254,  241150,  241314,  241315,  241478, 295927,  295928,  296090,  296091,  296092, 1713724, 1713725, 1713726, 1713727, 1713888, 4154531, 4154532, 4154694, 4154695, 4154696,
             4154859, 4154860, 4181589, 4181590, 4181591, 4181592,  377272,  377433,  377434,  377435,  377436,  1713889, 1713890, 1740621, 1740622, 1740623, 268538,  268866,  268867,  269031])

building = np.array([ 761701,  763506,  763670,  788762,  788763,
        788927,  923407,  950467, 1032467, 1032631, 1279778, 1279942,
       1306838, 1307002, 1388510, 2046964, 2047783, 2074024, 2098297,
       2098626, 2098790, 2125357, 2197525, 2197689, 2224585, 2224749,
       2660005, 2660006, 2661317, 2661481, 2688377, 2688541, 3262698,
       3262862, 3289758, 3289922, 3751416, 3751580, 3778476, 3778640])


In [ ]:
#Eval terrain
terreng_true = 0
terreng_tree_true = 0
terreng_building_true = 0

#eval tree
tree_true = 0
tree_terreng_true = 0
tree_building_true = 0

#eval 
building_true = 0
building_tree_true = 0
building_terreng_true = 0



for i in terrain_voxels:
    voxel = occupied_voxels[i]

    for true_voxel in terreng:
        if voxel == true_voxel:
            terreng_true += 1
            print("terreng ",voxel)

    for true_voxel in tree:
        if voxel == true_voxel:
            terreng_tree_true += 1

    for true_voxel in building:
        if voxel == true_voxel:
            print("building; ",voxel)
            terreng_building_true += 1


for i in tree_voxels:
    voxel = occupied_voxels[i]
    for true_voxel in tree:
        if voxel == true_voxel:
            tree_true += 1
            

    for true_voxel in building:
        if voxel == true_voxel:
            tree_building_true += 1

    for true_voxel in terreng:
        if voxel == true_voxel:
            tree_terreng_true += 1

for i in building_voxels:
    voxel = occupied_voxels[i]
    for true_voxel in building:
        if voxel == true_voxel:
            building_true += 1
        
    for true_voxel in terreng:
        if voxel == true_voxel:
            building_terreng_true += 1

    for true_voxel in tree:
        if voxel == true_voxel:
            building_tree_true += 1

        


In [ ]:
# print(F"Terreng {len(terreng)}, true: {terreng_true}, false tree {terreng_tree_true}, false building {terreng_building_true}")
# print(F"Tree {len(tree)}, true: {tree_true}, false terreng {tree_terreng_true}, false building {tree_building_true}")
# print(F"Building {len(building)}, true: {building_true} false terreng {building_terreng_true}, false tree {building_tree_true}")